# West Virginia Scraping 

In [53]:
# set up 
import pandas as pd 
import requests 
import bs4
import numpy as np
import time 
import re
from tika import parser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from io import BytesIO
from urllib.request import urlopen
from selenium.webdriver.chrome.options import Options

from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

%xmode Minimal

Exception reporting mode: Minimal


In [54]:
# # set headers
# header_list = {'User-Agent': REDACTED,
#               'Accept-Language': 'en-US,en;q=0.9'}

In [55]:
# state - UPDATE EACH TIME
state = 'west_virginia' 

In [56]:
# review of robots.txt - non-existent

In [ ]:
# test = pd.read_csv(f'{state}_2023.csv')
# test.head()

## Scrape links 
`Selenium`

In [108]:
driver = webdriver.Chrome()
driver.get('https://governor.wv.gov/News/press-releases/2023/Pages/default.aspx')

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'col-md-12')))

df_list = []

results = driver.find_elements(By.XPATH, "//div[@style='padding-bottom:15px;']")

# december is automatically loaded

for i in results:
    if len(i.text) > 0:
        text_element = i.text
        text = text_element.split('\n') 
        title = text[0]
        date = text[1] 
        link = i.find_element(By.TAG_NAME, 'a').get_attribute('href')
        df_list.append({'title': title, 'date': date, 'link': link})
    else:
        pass 

# remaining months 

for i in range(2, 13):
    month_button = driver.find_element(By.XPATH, f'//*[@id="accordion"]/div[{i}]/div[1]/h4/a')
    driver.execute_script("arguments[0].click();", month_button);

    time.sleep(5)

    results = driver.find_elements(By.XPATH, "//div[@style='padding-bottom:15px;']")

    for i in results:
        if len(i.text) > 0:
            text_element = i.text
            text = text_element.split('\n') 
            title = text[0]
            date = text[1] 
            link = i.find_element(By.TAG_NAME, 'a').get_attribute('href')
            #results_list = {'tite': title, 'date': date, 'link': link}
            df_list.append({'title': title, 'date': date, 'link': link})
        else:
            pass 
    
driver.close()

[<selenium.webdriver.remote.webelement.WebElement (session="8b31e8db411082a276791b96eeb814c8", element="D14AD65465038A3FA0AA9B7DFFBFB02F_element_11")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8b31e8db411082a276791b96eeb814c8", element="D14AD65465038A3FA0AA9B7DFFBFB02F_element_12")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8b31e8db411082a276791b96eeb814c8", element="D14AD65465038A3FA0AA9B7DFFBFB02F_element_13")>]

In [109]:
# combine into one df
combined = pd.DataFrame(df_list)
combined

,title,date,link
0,Gov. Justice issues proclamation declaring ext...,12/21/2023,https://governor.wv.gov/News/press-releases/20...
1,Gov. Justice awards $1 million grant to West V...,12/21/2023,https://governor.wv.gov/News/press-releases/20...
2,Gov. Justice announces Water Development Autho...,12/21/2023,https://governor.wv.gov/News/press-releases/20...
3,Gov. Justice orders flags to be flown at half-...,12/21/2023,https://governor.wv.gov/News/press-releases/20...
4,Gov. Justice issues proclamation declaring ext...,12/20/2023,https://governor.wv.gov/News/press-releases/20...
...,...,...,...
238,First Lady Justice announces two counties adde...,01/12/2023,https://governor.wv.gov/News/press-releases/20...
239,Gov. Justice delivers his 2023 West Virginia S...,01/11/2023,https://governor.wv.gov/News/press-releases/20...
240,Gov. Justice to deliver State of the State Add...,01/09/2023,https://governor.wv.gov/News/press-releases/20...
241,Gov. Justice announces appointment of longtime...,01/06/2023,https://governor.wv.gov/News/press-releases/20...


In [110]:
# create a blank content column 
combined['content'] = np.nan
combined.head()

,title,date,link,content
0,Gov. Justice issues proclamation declaring ext...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,NaN
1,Gov. Justice awards $1 million grant to West V...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,NaN
2,Gov. Justice announces Water Development Autho...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,NaN
3,Gov. Justice orders flags to be flown at half-...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,NaN
4,Gov. Justice issues proclamation declaring ext...,12/20/2023,https://governor.wv.gov/News/press-releases/20...,NaN


In [111]:
# export
combined.to_csv(f'{state}_links.csv', index = False)

In [112]:
test = pd.read_csv(f'{state}_links.csv')
test.head()

,title,date,link,content
0,Gov. Justice issues proclamation declaring ext...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,NaN
1,Gov. Justice awards $1 million grant to West V...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,NaN
2,Gov. Justice announces Water Development Autho...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,NaN
3,Gov. Justice orders flags to be flown at half-...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,NaN
4,Gov. Justice issues proclamation declaring ext...,12/20/2023,https://governor.wv.gov/News/press-releases/20...,NaN


## Scrape content from links
`Beautiful Soup`

### Test with one link

In [113]:
# test with one
link = 'https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-awards-$1-million-grant-to-West-Virginia-Chamber-Foundation,-boosting-Jobs-for-West-Virginia-Graduates-Program.aspx'
server_response = requests.get(link, headers = header_list)
server_response

<Response [200]>

In [125]:
# test with one contd 
soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
result = soup_link.find('td')
#result
string_result = [str(i.text) for i in result]
# string_result
string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
string_result_clean = [i for i in string_result_clean if len(i)>0] 
string_result_clean = [i.replace(u'\r', u'') for i in string_result_clean]
# string_result_clean
result_merged = " ".join(string_result_clean)
result_merged

"CHARLESTON, WV —Gov. Jim Justice today presented a $1 million check to the West Virginia Chamber Foundation, bolstering their Jobs for West Virginia’s Graduates (JWVG) program and securing brighter futures for countless students across the state.Gov. Justice joined Chamber Foundation representatives, program beneficiaries, and dedicated educators who champion JWVG’s transformative work for the ceremony at the State Capitol.                        “West Virginia students are our future,” Gov. Justice said. “This isn’t just a check, it’s a promise - a promise to bridge the gap between potential and accomplishment for our young people. JWVG’s proven track record in empowering students, particularly those facing challenges, makes it an investment worth celebrating.”The $1 million grant, included in the FY24 budget, passed by the West Virginia Legislature, and signed by Gov. Justice directly supports JWVG’s mission to equip high school students with the necessary skills and support to achi

In [ ]:
# define link scraping 
def get_wv_content(link, df, header_list): 
    try: 
        # check if content has already been pulled
        result = df.loc[df.link == link, 'content'].notnull().all()
        if result:
            return f'content already existing for {link}' 
        else: 
            try: 
                server_response = requests.get(link, headers = header_list)
                soup_link = bs4.BeautifulSoup(server_response.content, features="html.parser")
                result = soup_link.find('td')

                string_result = [str(i.text) for i in result]

                string_result_clean = [i.replace(u'\xa0', u'') for i in string_result]
                string_result_clean = [i.replace(u'\n', u'') for i in string_result_clean]
                string_result_clean = [i for i in string_result_clean if len(i)>0] 
                string_result_clean = [i.replace(u'\r', u'') for i in string_result_clean]

                result_merged = " ".join(string_result_clean)
                
                df.loc[df.link == link, 'content'] = result_merged
                return f"success: content added from {link}"
            except: 
                pass
    except Exception as e: 
        return f"unable to gather content from {link} due to {e}"
    #return df_updated 

In [131]:
# check one
get_wv_content(combined.link[0], combined, header_list) 
combined.content[0]

'content already existing for https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-issues-proclamation-declaring-extended-full-day-holiday-for-New-Years-for-public-employees.aspx'

'CHARLESTON, WV — Gov. Jim Justice has issued a proclamation, declaring Friday, Dec. 29, 2023, as a full-day state holiday for public employees.View the proclamation here.\u200bYesterday, Gov. Justice issued a proclamation, declaring Friday, Dec. 22, 2023, as a full-day state holiday for public employees.                        ###'

In [132]:
# check another
get_wv_content(combined.link[1], combined, header_list) 
combined.content[1] 

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-awards-$1-million-grant-to-West-Virginia-Chamber-Foundation,-boosting-Jobs-for-West-Virginia-Graduates-Program.aspx'

"CHARLESTON, WV —Gov. Jim Justice today presented a $1 million check to the West Virginia Chamber Foundation, bolstering their Jobs for West Virginia’s Graduates (JWVG) program and securing brighter futures for countless students across the state.Gov. Justice joined Chamber Foundation representatives, program beneficiaries, and dedicated educators who champion JWVG’s transformative work for the ceremony at the State Capitol.                        “West Virginia students are our future,” Gov. Justice said. “This isn’t just a check, it’s a promise - a promise to bridge the gap between potential and accomplishment for our young people. JWVG’s proven track record in empowering students, particularly those facing challenges, makes it an investment worth celebrating.”The $1 million grant, included in the FY24 budget, passed by the West Virginia Legislature, and signed by Gov. Justice directly supports JWVG’s mission to equip high school students with the necessary skills and support to achi

### Scrape content from all links 

In [133]:
for i in combined.link:
     get_wv_content(i, combined, header_list) 

'content already existing for https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-issues-proclamation-declaring-extended-full-day-holiday-for-New-Years-for-public-employees.aspx'

'content already existing for https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-awards-$1-million-grant-to-West-Virginia-Chamber-Foundation,-boosting-Jobs-for-West-Virginia-Graduates-Program.aspx'

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-announces-Water-Development-Authority-approves-over-$26-million-in-economic-enhancement-projects.aspx'

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-orders-flags-to-be-flown-at-half-staff-on-Saturday,-December-23,-in-honor-of-former-Delegate-Larry-Faircloth.aspx'

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-issues-proclamation-declaring-extended-full-day-holiday-for-Christmas-for-public-employees.aspx'

...

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/First-Lady-Justice-announces-two-counties-added-to-CIS-program;-three-new-schools-to-receive-Friends-With-Paws-therapy-dogs.aspx'

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-delivers-his-2023-West-Virginia-State-of-the-State-Address.aspx'

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-to-deliver-State-of-the-State-Address-Wednesday-night.aspx'

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-announces-appointment-of-longtime-Marion-County-educator-as-Executive-Director-of-School-Building-Authority.aspx'

'success: content added from https://governor.wv.gov/News/press-releases/2023/Pages/Gov.-Justice-says-another-month-of-revenue-surplus-means-major-tax-cuts-on-the-way.aspx'

In [134]:
combined.head()

,title,date,link,content
0,Gov. Justice issues proclamation declaring ext...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Jim Justice has issued a..."
1,Gov. Justice awards $1 million grant to West V...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV —Gov. Jim Justice today present..."
2,Gov. Justice announces Water Development Autho...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Justice announced today ..."
3,Gov. Justice orders flags to be flown at half-...,12/21/2023,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV – Gov. Jim Justice has issued a..."
4,Gov. Justice issues proclamation declaring ext...,12/20/2023,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Jim Justice has issued a..."


## Validate and Clean

In [135]:
#check that content was pulled accurately
combined[combined.content.isna()]

,title,date,link,content


In [136]:
# clean date column and subset to 2023 only 
combined['date_clean'] = pd.to_datetime(combined['date'])

combined_2023 = combined[(combined['date_clean'] >= '2023-01-01')
                     & (combined['date_clean'] <= '2023-12-31')].copy()

combined_2023 = combined_2023.drop(columns=['date']).copy()

In [137]:
combined_2023.head()
combined_2023.info()

,title,link,content,date_clean
0,Gov. Justice issues proclamation declaring ext...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Jim Justice has issued a...",2023-12-21
1,Gov. Justice awards $1 million grant to West V...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV —Gov. Jim Justice today present...",2023-12-21
2,Gov. Justice announces Water Development Autho...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Justice announced today ...",2023-12-21
3,Gov. Justice orders flags to be flown at half-...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV – Gov. Jim Justice has issued a...",2023-12-21
4,Gov. Justice issues proclamation declaring ext...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Jim Justice has issued a...",2023-12-20


<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   title       243 non-null    object        
 1   link        243 non-null    object        
 2   content     243 non-null    object        
 3   date_clean  243 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 9.5+ KB


In [138]:
print(f"pulled {str(len(combined_2023))} documents from {state} for 2023")

pulled 243 documents from west_virginia for 2023


## Export

In [139]:
combined_2023.to_csv(f'{state}_2023.csv', index = False)

In [140]:
test = pd.read_csv(f'{state}_2023.csv')
test.head()

,title,link,content,date_clean
0,Gov. Justice issues proclamation declaring ext...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Jim Justice has issued a...",2023-12-21
1,Gov. Justice awards $1 million grant to West V...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV —Gov. Jim Justice today present...",2023-12-21
2,Gov. Justice announces Water Development Autho...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Justice announced today ...",2023-12-21
3,Gov. Justice orders flags to be flown at half-...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV – Gov. Jim Justice has issued a...",2023-12-21
4,Gov. Justice issues proclamation declaring ext...,https://governor.wv.gov/News/press-releases/20...,"CHARLESTON, WV — Gov. Jim Justice has issued a...",2023-12-20
